In [44]:
!python3 -m pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable


In [45]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, balanced_accuracy_score, classification_report
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
from IPython.display import display


In [46]:
RANDOM_STATE   = 42
PREDICT_MONTH  = pd.Period('2023-07', 'M')   # month to predict
TRAIN_END   = pd.Period('2023-06', 'M')   # last month with labels for training
SHIFT_MACRO_BY = 1                           # shift macro features by +1M to avoid release leakage


In [47]:
stock = pd.read_csv("stock_data.csv")
index  = pd.read_csv("monashIndex.csv")
company  = pd.read_csv("company_info.csv")
vix    = pd.read_csv("vix_index.csv")
us10yt     = pd.read_csv("us_10_year_treasury.csv")
us5yt      = pd.read_csv("us_5_year_treasury.csv")
infl     = pd.read_csv("fed_inflation_rate.csv")
fedfunds = pd.read_csv("fed_funds_rate.csv")
unemp     = pd.read_csv("fed_unemployment_rate.csv")
train_tgt= pd.read_csv("training_targets.csv").astype({"stock_id":"str"})
test_tgt  = pd.read_csv("testing_targets.csv").astype({"stock_id":"str"})


In [48]:
# Sort by time for rolling/lag operations
def lag_and_roll(df, col, l1=True, mean3=True, time_col="month_id"):
    df = df.sort_values(time_col).copy()
    out = pd.DataFrame({time_col: df[time_col]})
    if l1:
        out[f"{col}_lag1"] = df[col].shift(1)
    if mean3:
        out[f"{col}_mean_3m"] = df[col].shift(1).rolling(3, min_periods=1).mean()
    return out

vix_feat = lag_and_roll(vix, "vix")  # vix_lag1, vix_mean_3m

yields = (us10yt.merge(us5yt, on="month_id", how="outer")
            .sort_values("month_id"))
yields_feat = yields[["month_id"]].copy()
yields_feat["y10_lag1"] = yields["10y_treasury"].shift(1)
yields_feat["y5_lag1"]  = yields["5y_treasury"].shift(1)
yields_feat["term_slope_lag1"] = yields_feat["y10_lag1"] - yields_feat["y5_lag1"]

fed = fedfunds.sort_values("month_id")
fed_feat = fed[["month_id"]].copy()
fed_feat["fed_rate_lag1"]   = fed["fed_rate"].shift(1)
# 3-month change (t-1 vs t-4)
fed_feat["fed_rate_chg_3m"] = fed["fed_rate"].shift(1) - fed["fed_rate"].shift(4)

cpi_feat = lag_and_roll(infl, "inflation_rate")  # inflation_rate_lag1, inflation_rate_mean_3m
cpi_feat = cpi_feat.rename(columns={
    "inflation_rate_lag1": "inflation_rate_lag1",
    "inflation_rate_mean_3m": "inflation_3m_mean"
})

unemp = unemp.sort_values("month_id")
unemp_feat = unemp[["month_id"]].copy()
unemp_feat["unemployment_rate_lag1"] = unemp["unemployment_rate"].shift(1)

# Index context
idx = index.sort_values("month_id")
idx_feat = idx[["month_id"]].copy()
idx_feat["index_return_lag1"]     = idx["index_return"].shift(1)
idx_feat["index_return_mean_3m"]  = idx["index_return"].shift(1).rolling(3, min_periods=1).mean()
# trailing 3m drawdown using index_value known by t-1
roll_max = idx["index_value"].shift(1).rolling(3, min_periods=1).max()
idx_feat["index_value_dd_3m"] = (idx["index_value"].shift(1) - roll_max) / roll_max

# Combine all macro + index features
macro_feat = (
    vix_feat.merge(yields_feat, on="month_id", how="outer")
            .merge(fed_feat, on="month_id", how="outer")
            .merge(cpi_feat, on="month_id", how="outer")
            .merge(unemp_feat, on="month_id", how="outer")
            .merge(idx_feat, on="month_id", how="outer")
            .sort_values("month_id")
)

print("macro_feat shape:", macro_feat.shape)


macro_feat shape: (42, 14)


In [49]:
stock = stock.sort_values(["stock_id", "month_id"]).copy()

# Pick the columns to lag (all time-varying)
to_lag = [
    "month_start_open_usd","month_end_close_usd",
    "month_high_usd","month_low_usd",
    "monthly_volume","intramonth_return","intramonth_volatility",
    "return_1m","return_3m","return_6m",
    "volatility_3m","volatility_6m",
    "trading_days","avg_volume_3m","volume_ratio","price_range_ratio"
]

missing_cols = [c for c in to_lag if c not in stock.columns]
if missing_cols:
    raise ValueError(f"Missing columns in stock_data.csv: {missing_cols}")

# Create *_lag1 versions aligned to the current month_id
lagged = stock.groupby("stock_id")[to_lag].shift(1)
lagged.columns = [f"{c}_lag1" for c in to_lag]

stock_feat = pd.concat([stock[["stock_id","month_id"]], lagged], axis=1)

# Drop the first available month per stock (no lag yet)
stock_feat = stock_feat.dropna(subset=[col for col in stock_feat.columns if col.endswith("_lag1")])

print("stock_feat shape:", stock_feat.shape)


stock_feat shape: (24867, 18)


In [50]:
# Ensure company_ohe exists (one-hot of company_info)
if 'company_ohe' not in globals():
    cat_cols = ['sector','business_model','geographic_focus','business_maturity','competitive_position',
                'market_cap_category','revenue_tier','profitability_profile','asset_intensity','financial_strength']
    comp = company[['stock_id'] + cat_cols].copy()
    company_ohe = pd.get_dummies(comp, columns=cat_cols, prefix=cat_cols, prefix_sep='=')

features_panel = (
    stock_feat
    .merge(macro_feat, on="month_id", how="left")
    .merge(company_ohe, on="stock_id", how="left")
    .sort_values(["stock_id","month_id"])
)

print("features_panel shape:", features_panel.shape)
features_panel.head(3)


features_panel shape: (24867, 70)


,stock_id,month_id,month_start_open_usd_lag1,month_end_close_usd_lag1,month_high_usd_lag1,month_low_usd_lag1,monthly_volume_lag1,intramonth_return_lag1,intramonth_volatility_lag1,return_1m_lag1,...,revenue_tier=Tier_3,profitability_profile=High_Margin,profitability_profile=Low_Margin,profitability_profile=Standard,asset_intensity=Asset_Light,asset_intensity=Capital_Intensive,asset_intensity=Moderate,financial_strength=Developing,financial_strength=Stable,financial_strength=Strong
0,US001,2020_02,120.192,107.326,123.486,107.035,84539259.0,-0.107045,0.253304,-0.100669,...,False,False,False,True,False,False,True,False,True,False
1,US001,2020_03,108.252,101.868,111.641,99.636,91313882.0,-0.058974,0.259208,-0.050854,...,False,False,False,True,False,False,True,False,True,False
2,US001,2020_04,103.301,93.178,105.622,77.841,177930833.0,-0.097995,0.892709,-0.085299,...,False,False,False,True,False,False,True,False,True,False


In [51]:
macro = (
    vix[["month_id", "vix"]]
    .merge(us10yt[["month_id", "10y_treasury"]], on="month_id", how="outer")
    .merge(us5yt[["month_id", "5y_treasury"]], on="month_id", how="outer")
    .merge(infl[["month_id", "inflation_rate"]], on="month_id", how="outer")
    .merge(fedfunds[["month_id", "fed_rate"]], on="month_id", how="outer")
    .merge(unemp[["month_id", "unemployment_rate"]], on="month_id", how="outer")
    .merge(index[["month_id", "index_return", "index_value"]], on="month_id", how="outer")
    .sort_values("month_id")
)

# engineered macro fields
macro["TERM_SPREAD"] = macro["10y_treasury"] - macro["5y_treasury"]
macro["REAL_RATE_PROXY"] = macro["10y_treasury"] - macro["inflation_rate"]
for col in ["vix", "10y_treasury", "5y_treasury", "inflation_rate", "fed_rate", "unemployment_rate"]:
    macro[f"{col}_CHG_1M"] = macro[col].diff()

# lag ALL macro features by 1 month to avoid leak
macro_lag = macro.copy()
macro_cols = [c for c in macro.columns if c != "month_id"]
macro_lag[macro_cols] = macro_lag[macro_cols].shift(1)
macro_lag = macro_lag.rename(columns={c: f"{c}_lag1" for c in macro_cols})


In [52]:
stock = stock.sort_values(["stock_id", "month_id"])

In [53]:
to_lag = [
    "return_1m","return_3m","return_6m",
    "intramonth_volatility","volatility_3m","volatility_6m",
    "volume_ratio","avg_volume_3m","monthly_volume",
    "price_range_ratio","trading_days"
]
to_lag = [c for c in to_lag if c in stock.columns]

for c in to_lag:
    stock[f"{c}_lag1"] = stock.groupby("stock_id")[c].shift(1)

lag_feats = sorted([f"{c}_lag1" for c in to_lag if f"{c}_lag1" in stock.columns])

# Build panel of features only (lagged stock + lagged macro) + company info
panel = stock[["stock_id","month_id"] + lag_feats].merge(macro_lag, on="month_id", how="left")
panel = panel.merge(company, on="stock_id", how="left")

# Add training labels by merge (do NOT compute from prices here)
panel = panel.merge(
    train_tgt[["stock_id","month_id","outperform_binary"]],
    on=["stock_id","month_id"], how="left"
)


In [54]:
# ---- Normalize month_id safely to Period('M') handling 'YYYY_MM' format ----
def force_period_m(df, col="month_id"):
    if col in df.columns:
        s = df[col].astype(str).str.strip()
        # First try explicit 'YYYY_MM' format
        p = pd.to_datetime(s, format="%Y_%m", errors="coerce")
        # Fallback: accept 'YYYY-MM' or other parseable forms
        m = p.isna()
        if m.any():
            p[m] = pd.to_datetime(s[m].str.replace('_', '-', regex=False), format="%Y-%m", errors="coerce")
        df[col] = p.dt.to_period("M")
    return df

stock     = force_period_m(stock)
macro_lag = force_period_m(macro_lag)
train_tgt = force_period_m(train_tgt)
test_tgt  = force_period_m(test_tgt)
panel     = force_period_m(panel)

# (optional sanity) - Removed print statement causing NameError
# print("DTypes:",
#       "panel.month_id ->", panel["month_id"].dtype,
#       "| tjm1 type ->", type(tjm1))

In [55]:
train_df = (train_tgt
            .merge(panel, on=["stock_id","month_id"], how="inner")
            .sort_values(["stock_id","month_id"]))

# Build July 2023 features from June 2023 data (align lag=1 to prediction month)
prev_m = PREDICT_MONTH - 1
# Stock features for prev_m, renamed to *_lag1 and assigned to month_id=PREDICT_MONTH
stock_prev = stock.loc[stock["month_id"] == prev_m, ['stock_id'] + to_lag].copy()
stock_prev = stock_prev.rename(columns={c: f'{c}_lag1' for c in to_lag})
stock_prev['month_id'] = PREDICT_MONTH

# Macro features for prev_m, rename all non-id columns to *_lag1 and assign to PREDICT_MONTH
macro_prev = macro.loc[macro["month_id"] == prev_m].copy()
macro_prev = macro_prev.rename(columns={c: f'{c}_lag1' for c in macro_prev.columns if c != 'month_id'})
macro_prev['month_id'] = PREDICT_MONTH

panel_pred = (stock_prev
               .merge(macro_prev, on='month_id', how='left')
               .merge(company, on='stock_id', how='left'))

test_df  = (test_tgt
            .merge(panel_pred, on=["stock_id","month_id"], how="left")
            .sort_values(["stock_id","month_id"]))

print(train_df["month_id"].min(), train_df["month_id"].max())
print("Unique test months:", sorted(test_df["month_id"].unique().astype(str)))

# Feature columns = everything except identifiers & labels
# Adjusted to exclude the merged target columns
drop_cols = ["stock_id","month_id","outperform_binary_x","outperform_binary_y","test_outperform","excess_return"]
feat_cols = [c for c in train_df.columns if c not in drop_cols]

X_train = train_df[feat_cols].replace([np.inf, -np.inf], np.nan)
# Use the correctly named target column from the merge for y_train
y_train = train_df["outperform_binary_x"].astype(int)

X_test  = test_df[feat_cols].replace([np.inf, -np.inf], np.nan)
print("X_test NA fraction BEFORE fill:", X_test.isna().mean().mean())
print("Top 10 X_test NA columns BEFORE fill:")
print(X_test.isna().mean().sort_values(ascending=False).head(10))

# Consistent NA imputation from TRAIN medians
train_medians = X_train.median(numeric_only=True)
X_train = X_train.fillna(train_medians)
X_test  = X_test.fillna(train_medians)

# Define numeric and categorical columns based on X_train
numeric_cols = X_train.select_dtypes(include=np.number).columns.tolist()
categorical_cols = X_train.select_dtypes(include='object').columns.tolist()

print("X_train/test shapes:", X_train.shape, X_test.shape)
print("Numeric columns:", numeric_cols)
print("Categorical columns:", categorical_cols)

2020-01 2023-06
Unique test months: ['2023-07']
X_test NA fraction BEFORE fill: 0.43243243243243246
Top 10 X_test NA columns BEFORE fill:
index_value_lag1                 1.0
5y_treasury_lag1                 1.0
5y_treasury_CHG_1M_lag1          1.0
10y_treasury_CHG_1M_lag1         1.0
vix_CHG_1M_lag1                  1.0
REAL_RATE_PROXY_lag1             1.0
TERM_SPREAD_lag1                 1.0
unemployment_rate_CHG_1M_lag1    1.0
index_return_lag1                1.0
unemployment_rate_lag1           1.0
dtype: float64
X_train/test shapes: (25618, 37) (616, 37)
Numeric columns: ['avg_volume_3m_lag1', 'intramonth_volatility_lag1', 'monthly_volume_lag1', 'price_range_ratio_lag1', 'return_1m_lag1', 'return_3m_lag1', 'return_6m_lag1', 'trading_days_lag1', 'volatility_3m_lag1', 'volatility_6m_lag1', 'volume_ratio_lag1', 'vix_lag1', '10y_treasury_lag1', '5y_treasury_lag1', 'inflation_rate_lag1', 'fed_rate_lag1', 'unemployment_rate_lag1', 'index_return_lag1', 'index_value_lag1', 'TERM_SPREAD_la

In [56]:
print("Panel month range:", panel["month_id"].min(), "→", panel["month_id"].max())
print("Counts by month (tail):")
print(panel["month_id"].value_counts().sort_index().tail(6))

missing_cols = [c for c in ["monthly_volume","avg_volume_3m","volume_ratio","price_range_ratio","trading_days"]
                if c not in stock.columns]
if missing_cols:
    print("Missing in stock_data.csv:", missing_cols)

Panel month range: 2020-01 → 2023-06
Counts by month (tail):
month_id
2023-01    616
2023-02    616
2023-03    616
2023-04    616
2023-05    616
2023-06    616
Freq: M, Name: count, dtype: int64


MODELO 1

In [57]:
# ===== Model A: Logistic Regression (with scaling) =====
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, accuracy_score, balanced_accuracy_score, f1_score

# preprocessors
num_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

try:
    ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
except TypeError:  # older sklearn
    ohe = OneHotEncoder(handle_unknown="ignore", sparse=False)

cat_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("ohe", ohe)
])

pre = ColumnTransformer(
    transformers=[
        ("num", num_pipe, numeric_cols),
        ("cat", cat_pipe, categorical_cols),
    ],
    remainder="drop",
    verbose_feature_names_out=False,
)

clf = LogisticRegression(
    max_iter=2000,
    class_weight="balanced",
    C=1.0,
    random_state=RANDOM_STATE
)

pipe_lr = Pipeline([
    ("pre", pre),
    ("clf", clf)
])

print("X_train/X_test shapes:", X_train.shape, X_test.shape)
pipe_lr.fit(X_train, y_train)

# July 2023 predictions
proba = pipe_lr.predict_proba(X_test)[:, 1]
pred  = (proba >= 0.5).astype(int)

preds_lr = test_df[["stock_id","month_id"]].copy()
preds_lr["proba"] = proba
preds_lr["pred"] = pred

# If ground truth for July exists, print quick metrics
y_test_jul = globals().get('y_test_jul', None)
if y_test_jul is not None and y_test_jul.notna().any():
    y_true = y_test_jul.values.astype(int)
    print("LR — July 2023")
    print("  AUC:                ", round(roc_auc_score(y_true, proba), 4))
    print("  Accuracy:           ", round(accuracy_score(y_true, pred), 4))
    print("  Balanced Accuracy:  ", round(balanced_accuracy_score(y_true, pred), 4))
    print("  F1 (positive=1):    ", round(f1_score(y_true, pred), 4))

# Top candidates
preds_lr.sort_values("proba", ascending=False).head(10)


X_train/X_test shapes: (25618, 37) (616, 37)


/Users/valentinaovallevelandia/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_linear_loss.py:200: RuntimeWarning: divide by zero encountered in matmul
  raw_prediction = X @ weights + intercept
/Users/valentinaovallevelandia/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_linear_loss.py:200: RuntimeWarning: overflow encountered in matmul
  raw_prediction = X @ weights + intercept
/Users/valentinaovallevelandia/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_linear_loss.py:200: RuntimeWarning: invalid value encountered in matmul
  raw_prediction = X @ weights + intercept
/Users/valentinaovallevelandia/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_linear_loss.py:330: RuntimeWarning: divide by zero encountered in matmul
  grad[:n_features] = X.T @ grad_pointwise + l2_reg_strength * weights
/Users/valentinaovallevelandia/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_linear_loss.py:330: RuntimeWar

,stock_id,month_id,proba,pred
357,US360,2023-07,0.563992,1
423,US427,2023-07,0.562061,1
344,US347,2023-07,0.548132,1
358,US361,2023-07,0.543838,1
86,US087,2023-07,0.539039,1
436,US440,2023-07,0.536951,1
198,US199,2023-07,0.529373,1
127,US128,2023-07,0.529369,1
193,US194,2023-07,0.528148,1
71,US072,2023-07,0.527883,1


In [58]:
display(X_train)
display(X_test)

,avg_volume_3m_lag1,intramonth_volatility_lag1,monthly_volume_lag1,price_range_ratio_lag1,return_1m_lag1,return_3m_lag1,return_6m_lag1,trading_days_lag1,volatility_3m_lag1,volatility_6m_lag1,...,sector,business_model,geographic_focus,business_maturity,competitive_position,market_cap_category,revenue_tier,profitability_profile,asset_intensity,financial_strength
0,2040278.5,0.315078,44100050.0,0.143176,0.007910,0.025938,0.054107,21.0,0.276432,0.300568,...,Industrials,B2B,Domestic,Mature,Market_Leader,Large,Tier_1,Standard,Moderate,Stable
1,3182487.0,0.253304,84539259.0,0.136875,-0.100669,-0.030041,-0.075676,21.0,0.278396,0.201608,...,Industrials,B2B,Domestic,Mature,Market_Leader,Large,Tier_1,Standard,Moderate,Stable
2,3686371.0,0.259208,91313882.0,0.110905,-0.050854,-0.112968,-0.060752,19.0,0.245523,0.192782,...,Industrials,B2B,Domestic,Mature,Market_Leader,Large,Tier_1,Standard,Moderate,Stable
3,5360363.0,0.892709,177930833.0,0.268931,-0.085299,-0.219214,-0.154874,22.0,0.088364,0.213051,...,Industrials,B2B,Domestic,Mature,Market_Leader,Large,Tier_1,Standard,Moderate,Stable
4,5571115.0,0.408352,98448862.0,0.242954,0.112885,-0.033810,-0.062835,21.0,0.366808,0.291241,...,Industrials,B2B,Domestic,Mature,Market_Leader,Large,Tier_1,Standard,Moderate,Stable
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25613,386326.0,0.335883,10455200.0,0.134183,0.056930,-0.077414,0.053293,20.0,0.251799,0.496076,...,Finance,B2C,Domestic,Mature,Strong_Player,Mid,Tier_2,Low_Margin,Asset_Light,Stable
25614,380447.0,0.238944,6730100.0,0.065131,-0.003958,0.001408,0.110988,19.0,0.183777,0.483301,...,Finance,B2C,Domestic,Mature,Strong_Player,Mid,Tier_2,Low_Margin,Asset_Light,Stable
25615,767948.0,0.483441,33499300.0,0.213917,-0.141146,-0.095845,0.018786,23.0,0.351460,0.527833,...,Finance,B2C,Domestic,Mature,Strong_Player,Mid,Tier_2,Low_Margin,Asset_Light,Stable
25616,851485.0,0.251317,15968600.0,0.109712,-0.069057,-0.203621,-0.265272,19.0,0.237719,0.236223,...,Finance,B2C,Domestic,Mature,Strong_Player,Mid,Tier_2,Low_Margin,Asset_Light,Stable


,avg_volume_3m_lag1,intramonth_volatility_lag1,monthly_volume_lag1,price_range_ratio_lag1,return_1m_lag1,return_3m_lag1,return_6m_lag1,trading_days_lag1,volatility_3m_lag1,volatility_6m_lag1,...,sector,business_model,geographic_focus,business_maturity,competitive_position,market_cap_category,revenue_tier,profitability_profile,asset_intensity,financial_strength
0,4070595.0,0.394990,113707426,0.133839,0.072661,-0.033285,-0.141454,21,0.318319,0.211518,...,Industrials,B2B,Domestic,Mature,Market_Leader,Large,Tier_1,Standard,Moderate,Stable
1,986056.0,0.254188,17775400,0.144616,0.138255,0.057193,0.283544,21,0.361340,0.345485,...,Industrials,B2B,Global,Growth,Strong_Player,Mid,Tier_2,Standard,Capital_Intensive,Stable
2,4730170.0,0.144965,108476000,0.086351,0.068823,0.082015,0.002468,21,0.322137,0.251718,...,Healthcare,B2C,Global,Mature,Market_Leader,Large,Tier_1,High_Margin,Moderate,Stable
3,5385544.0,0.205950,135187500,0.064395,-0.023413,-0.146832,-0.150659,21,0.112949,0.189711,...,Healthcare,B2B,Global,Mature,Market_Leader,Large,Tier_1,High_Margin,Asset_Light,Strong
4,2527162.0,0.238028,62201300,0.104439,0.008695,0.083908,0.165713,21,0.194211,0.189313,...,Technology,B2B,Global,Mature,Market_Leader,Large,Tier_1,High_Margin,Moderate,Strong
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
611,439167.0,0.521860,8879900,0.212833,0.081382,0.061148,-0.263401,21,0.404162,0.617049,...,Finance,B2C,Domestic,Cyclical,Niche_Specialist,Mid,Tier_2,Standard,Moderate,Developing
612,214339.0,0.376659,4807000,0.132246,0.010794,-0.118542,-0.231856,21,0.220072,0.193215,...,Finance,B2C,International,Mature,Strong_Player,Mid,Tier_2,Standard,Moderate,Developing
613,318920.0,0.488021,7278500,0.211115,0.100889,-0.057068,-0.195795,21,0.350894,0.380617,...,Finance,B2C,Domestic,Growth,Strong_Player,Mid,Tier_2,Standard,Moderate,Developing
614,90620.0,0.385755,2342700,0.140510,0.092747,0.112078,0.053035,21,0.236120,0.226100,...,Finance,B2C,International,Mature,Strong_Player,Mid,Tier_2,Low_Margin,Moderate,Developing


MODEL 2

In [59]:
# ===== Model B: Random Forest (no scaling needed) =====
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, balanced_accuracy_score, f1_score

# Preprocessing for trees: impute only + OHE
num_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
])

try:
    ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
except TypeError:
    ohe = OneHotEncoder(handle_unknown="ignore", sparse=False)

cat_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("ohe", ohe)
])

pre_tree = ColumnTransformer(
    transformers=[
        ("num", num_pipe, numeric_cols),
        ("cat", cat_pipe, categorical_cols),
    ],
    remainder="drop",
    verbose_feature_names_out=False,
)

rf = RandomForestClassifier(
    n_estimators=600,
    max_depth=None,
    min_samples_leaf=2,
    class_weight="balanced_subsample",
    random_state=RANDOM_STATE,
    n_jobs=-1
)

pipe_rf = Pipeline([
    ("pre", pre_tree),
    ("clf", rf)
])

pipe_rf.fit(X_train, y_train)

proba = pipe_rf.predict_proba(X_test)[:, 1]
pred  = (proba >= 0.5).astype(int)

preds_rf = test_df[["stock_id","month_id"]].copy()
preds_rf["proba"] = proba
preds_rf["pred"] = pred

y_test_jul = globals().get('y_test_jul', None)
if y_test_jul is not None and y_test_jul.notna().any():
    y_true = y_test_jul.values.astype(int)
    print("RF — July 2023")
    print("  AUC:                ", round(roc_auc_score(y_true, proba), 4))
    print("  Accuracy:           ", round(accuracy_score(y_true, pred), 4))
    print("  Balanced Accuracy:  ", round(balanced_accuracy_score(y_true, pred), 4))
    print("  F1 (positive=1):    ", round(f1_score(y_true, pred), 4))

preds_rf.sort_values("proba", ascending=False).head(10)


,stock_id,month_id,proba,pred
48,US049,2023-07,0.677878,1
491,US496,2023-07,0.658666,1
139,US140,2023-07,0.654829,1
253,US255,2023-07,0.653749,1
65,US066,2023-07,0.649890,1
361,US364,2023-07,0.607100,1
200,US201,2023-07,0.602688,1
498,US503,2023-07,0.599577,1
150,US151,2023-07,0.598330,1
509,US514,2023-07,0.596790,1


MODELO 3

In [60]:
# ===== Model C: HistGradientBoosting (tree boosting) =====
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.experimental import enable_hist_gradient_boosting  # noqa: F401
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, balanced_accuracy_score, f1_score

# Preprocess: impute + dense OHE (HGB needs dense)
num_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
])

try:
    ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
except TypeError:
    ohe = OneHotEncoder(handle_unknown="ignore", sparse=False)

cat_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("ohe", ohe)
])

pre_hgb = ColumnTransformer(
    transformers=[
        ("num", num_pipe, numeric_cols),
        ("cat", cat_pipe, categorical_cols),
    ],
    remainder="drop",
    verbose_feature_names_out=False,
)

hgb = HistGradientBoostingClassifier(
    learning_rate=0.06,
    max_depth=7,            # None also works; 7 is a good start
    max_iter=400,
    min_samples_leaf=30,
    l2_regularization=0.0,
    random_state=RANDOM_STATE
)

pipe_hgb = Pipeline([
    ("pre", pre_hgb),
    ("clf", hgb)
])

pipe_hgb.fit(X_train, y_train)

proba = pipe_hgb.predict_proba(X_test)[:, 1]
pred  = (proba >= 0.5).astype(int)

preds_hgb = test_df[["stock_id","month_id"]].copy()
preds_hgb["proba"] = proba
preds_hgb["pred"] = pred

y_test_jul = globals().get('y_test_jul', None)
if y_test_jul is not None and y_test_jul.notna().any():
    y_true = y_test_jul.values.astype(int)
    print("HGB — July 2023")
    print("  AUC:                ", round(roc_auc_score(y_true, proba), 4))
    print("  Accuracy:           ", round(accuracy_score(y_true, pred), 4))
    print("  Balanced Accuracy:  ", round(balanced_accuracy_score(y_true, pred), 4))
    print("  F1 (positive=1):    ", round(f1_score(y_true, pred), 4))

preds_hgb.sort_values("proba", ascending=False).head(10)


,stock_id,month_id,proba,pred
342,US345,2023-07,0.660224,1
436,US440,2023-07,0.654939,1
86,US087,2023-07,0.649977,1
401,US404,2023-07,0.644513,1
500,US505,2023-07,0.636936,1
450,US454,2023-07,0.630214,1
370,US373,2023-07,0.629840,1
7,US008,2023-07,0.619474,1
341,US344,2023-07,0.609419,1
517,US522,2023-07,0.595939,1


In [61]:
def merge_preds(meta, **pred_dfs):
    base = meta[["stock_id","month_id"]].drop_duplicates().copy()
    for name, dfp in pred_dfs.items():
        if dfp is not None:
            base = base.merge(
                dfp[["stock_id","proba"]].rename(columns={"proba": f"proba_{name}"}),
                on="stock_id", how="left"
            )
    return base

preds_all = merge_preds(
    test_df[["stock_id","month_id"]],
    lr=preds_lr if "preds_lr" in globals() else None,
    rf=preds_rf if "preds_rf" in globals() else None,
    hgb=preds_hgb if "preds_hgb" in globals() else None,
)

proba_cols = [c for c in preds_all.columns if c.startswith("proba_")]
preds_all["mean_proba"] = preds_all[proba_cols].mean(axis=1, skipna=True)
preds_all["vote_1s"]    = (preds_all[proba_cols] >= 0.5).sum(axis=1)
preds_all["pred_ens"]   = (preds_all["mean_proba"] >= 0.5).astype(int)

print("Rows in July universe:", len(preds_all))
preds_all.sort_values("mean_proba", ascending=False).head(10)

Rows in July universe: 616


,stock_id,month_id,proba_lr,proba_rf,proba_hgb,mean_proba,vote_1s,pred_ens
48,US049,2023-07,0.508503,0.677878,0.535887,0.574089,3,1
200,US201,2023-07,0.507161,0.602688,0.589040,0.566296,3,1
253,US255,2023-07,0.499346,0.653749,0.531990,0.561695,2,1
342,US345,2023-07,0.472016,0.551880,0.660224,0.561374,2,1
500,US505,2023-07,0.515665,0.521113,0.636936,0.557904,3,1
436,US440,2023-07,0.536951,0.480590,0.654939,0.557493,2,1
401,US404,2023-07,0.527838,0.494021,0.644513,0.555458,2,1
7,US008,2023-07,0.470432,0.572070,0.619474,0.553992,2,1
139,US140,2023-07,0.463742,0.654829,0.542752,0.553775,2,1
79,US080,2023-07,0.513108,0.579930,0.561468,0.551502,3,1


In [62]:
def eval_on_july(name, proba, y_true):
    m = ~np.isnan(proba)
    y = y_true[m].astype(int)
    p = proba[m]
    pred = (p >= 0.5).astype(int)
    print(f"{name:>6} | AUC={roc_auc_score(y,p):.3f}  Acc={accuracy_score(y,pred):.3f}  "
          f"BalAcc={balanced_accuracy_score(y,pred):.3f}  F1={f1_score(y,pred):.3f}  n={m.sum()}")

if y_test_jul is not None and y_test_jul.notna().any():
    y_true = y_test_jul.values
    for c in proba_cols:
        eval_on_july(c.replace("proba_","").upper(), preds_all[c].values, y_true)
    eval_on_july("ENSEMB", preds_all["mean_proba"].values, y_true)


In [63]:
# Ranked picks by ensemble probability
ranked = preds_all.sort_values(["mean_proba","vote_1s"], ascending=[False, False]).reset_index(drop=True)

# Choose a portfolio size (e.g., top 10%); feel free to change
k = max(1, int(0.10 * len(ranked)))
top_k = ranked.head(k).copy()

# Join optional metadata for reporting
if "company" in globals():
    top_k = top_k.merge(company[["stock_id","sector"]], on="stock_id", how="left")

print("Top-k (k=", k, ") preview:")
display(top_k.head(15))

# If ground truth & excess returns available, grade the portfolio:
tj = test_df["month_id"].iloc[0]
try:
    jul_truth = (test_tgt.loc[test_tgt["month_id"] == tj, ["stock_id","outperform_binary","excess_return"]]
                         .drop_duplicates("stock_id"))
except Exception:
    jul_truth = None

if jul_truth is not None and not jul_truth.empty:
    top_eval = top_k.merge(jul_truth, on="stock_id", how="left")
    hit_rate = np.nanmean(top_eval["outperform_binary"])
    avg_excess = np.nanmean(top_eval["excess_return"])
    print(f"Top-{k} hit rate: {hit_rate:.3f} | mean excess return: {avg_excess:.3%}")

    # Bottom decile baseline
    bot_k = ranked.tail(k).merge(jul_truth, on="stock_id", how="left")
    bot_hit = np.nanmean(bot_k["outperform_binary"])
    bot_excess = np.nanmean(bot_k["excess_return"])
    print(f"Bottom-{k} hit rate: {bot_hit:.3f} | mean excess return: {bot_excess:.3%}")
    if "sector" in top_k.columns:
        print("\nTop-k sector mix (% of names):")
        print((top_k["sector"].value_counts(normalize=True)*100).round(1).to_string())


Top-k (k= 61 ) preview:


,stock_id,month_id,proba_lr,proba_rf,proba_hgb,mean_proba,vote_1s,pred_ens,sector
0,US049,2023-07,0.508503,0.677878,0.535887,0.574089,3,1,Technology
1,US201,2023-07,0.507161,0.602688,0.589040,0.566296,3,1,Technology
2,US255,2023-07,0.499346,0.653749,0.531990,0.561695,2,1,Technology
3,US345,2023-07,0.472016,0.551880,0.660224,0.561374,2,1,Utilities
4,US505,2023-07,0.515665,0.521113,0.636936,0.557904,3,1,Technology
5,US440,2023-07,0.536951,0.480590,0.654939,0.557493,2,1,Consumer_Discretionary
6,US404,2023-07,0.527838,0.494021,0.644513,0.555458,2,1,Consumer_Discretionary
7,US008,2023-07,0.470432,0.572070,0.619474,0.553992,2,1,Utilities
8,US140,2023-07,0.463742,0.654829,0.542752,0.553775,2,1,Technology
9,US080,2023-07,0.513108,0.579930,0.561468,0.551502,3,1,Technology


Top-61 hit rate: nan | mean excess return: nan%
Bottom-61 hit rate: nan | mean excess return: nan%

Top-k sector mix (% of names):
sector
Technology                42.6
Utilities                 24.6
Real_Estate               11.5
Consumer_Discretionary     9.8
Finance                    8.2
Healthcare                 1.6
Industrials                1.6


/var/folders/8g/2nwmz93j45s_r3g3lzds5fmr0000gn/T/ipykernel_21109/4062268398.py:25: RuntimeWarning: Mean of empty slice
  hit_rate = np.nanmean(top_eval["outperform_binary"])
/var/folders/8g/2nwmz93j45s_r3g3lzds5fmr0000gn/T/ipykernel_21109/4062268398.py:26: RuntimeWarning: Mean of empty slice
  avg_excess = np.nanmean(top_eval["excess_return"])
/var/folders/8g/2nwmz93j45s_r3g3lzds5fmr0000gn/T/ipykernel_21109/4062268398.py:31: RuntimeWarning: Mean of empty slice
  bot_hit = np.nanmean(bot_k["outperform_binary"])
/var/folders/8g/2nwmz93j45s_r3g3lzds5fmr0000gn/T/ipykernel_21109/4062268398.py:32: RuntimeWarning: Mean of empty slice
  bot_excess = np.nanmean(bot_k["excess_return"])


In [64]:
# Random Forest importances
try:
    rf_feats = list(pipe_rf.named_steps["pre"].get_feature_names_out())
    rf_imps  = pipe_rf.named_steps["clf"].feature_importances_
    feat_imp_rf = (pd.DataFrame({"feature": rf_feats, "importance": rf_imps})
                    .sort_values("importance", ascending=False)
                    .head(25))
    display(feat_imp_rf)
except Exception as e:
    print("RF importance not available:", e)

# HistGB importances (if exposed)
try:
    hgb_feats = list(pipe_hgb.named_steps["pre"].get_feature_names_out())
    hgb_imps  = getattr(pipe_hgb.named_steps["clf"], "feature_importances_", None)
    if hgb_imps is not None:
        feat_imp_hgb = (pd.DataFrame({"feature": hgb_feats, "importance": hgb_imps})
                         .sort_values("importance", ascending=False)
                         .head(25))
        display(feat_imp_hgb)
except Exception as e:
    print("HGB importance not available:", e)


,feature,importance
1,intramonth_volatility_lag1,0.068487
5,return_3m_lag1,0.065601
6,return_6m_lag1,0.064612
3,price_range_ratio_lag1,0.063026
9,volatility_6m_lag1,0.060934
4,return_1m_lag1,0.060618
8,volatility_3m_lag1,0.057984
10,volume_ratio_lag1,0.056526
2,monthly_volume_lag1,0.054333
0,avg_volume_3m_lag1,0.054151


In [65]:
# ===== Advanced Task: Regression for July 2023 Excess Return =====
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, r2_score

# Continuous target: use training excess_return
if 'excess_return' in train_df.columns:
    y_reg = train_df['excess_return'].astype(float)
else:
    # Fallback if suffixed (unlikely here)
    y_cols = [c for c in train_df.columns if c.startswith('excess_return')]
    assert len(y_cols) >= 1, 'excess_return not found in training data'
    y_reg = train_df[y_cols[0]].astype(float)

# Preprocessor (shared)
num_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()),
])
try:
    ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
except TypeError:
    ohe = OneHotEncoder(handle_unknown="ignore", sparse=False)
cat_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("ohe", ohe),
])

pre_reg = ColumnTransformer(
    transformers=[
        ("num", num_pipe, numeric_cols),
        ("cat", cat_pipe, categorical_cols),
    ],
    remainder="drop",
    verbose_feature_names_out=False,
)

# Models
ridge = Ridge(alpha=1.0)
rfr = RandomForestRegressor(
    n_estimators=400,
    max_depth=None,
    min_samples_leaf=2,
    random_state=RANDOM_STATE,
    n_jobs=-1,
)
hgbr = HistGradientBoostingRegressor(
    loss="squared_error",
    learning_rate=0.06,
    max_depth=7,
    max_iter=300,
    min_samples_leaf=30,
    l2_regularization=0.0,
    random_state=RANDOM_STATE,
)

pipe_ridge = Pipeline([("pre", pre_reg), ("reg", ridge)])
pipe_rfr   = Pipeline([("pre", pre_reg), ("reg", rfr)])
pipe_hgbr  = Pipeline([("pre", pre_reg), ("reg", hgbr)])

# Fit
pipe_ridge.fit(X_train, y_reg)
pipe_rfr.fit(X_train, y_reg)
pipe_hgbr.fit(X_train, y_reg)

# Quick in-sample fit metrics (orientation only)
for name, pipe in [("Ridge", pipe_ridge), ("RF", pipe_rfr), ("HGBR", pipe_hgbr)]:
    pred_tr = pipe.predict(X_train)
    print(f"{name:>5} | R2={r2_score(y_reg, pred_tr):.3f}  MAE={mean_absolute_error(y_reg, pred_tr):.5f}")

# July 2023 predictions (continuous excess return)
reg_ridge = test_df[["stock_id","month_id"]].copy()
reg_ridge["pred_excess_return"] = pipe_ridge.predict(X_test)

reg_rfr = test_df[["stock_id","month_id"]].copy()
reg_rfr["pred_excess_return"] = pipe_rfr.predict(X_test)

reg_hgbr = test_df[["stock_id","month_id"]].copy()
reg_hgbr["pred_excess_return"] = pipe_hgbr.predict(X_test)

# Ensemble mean of the three regressors
reg_all = test_df[["stock_id","month_id"]].copy()
reg_all["ridge"] = reg_ridge["pred_excess_return"].values
reg_all["rf"]    = reg_rfr["pred_excess_return"].values
reg_all["hgbr"]  = reg_hgbr["pred_excess_return"].values
reg_all["mean_pred_excess"] = reg_all[["ridge","rf","hgbr"]].mean(axis=1)

print("\nJuly 2023 regression predictions (head):")
display(reg_all.head(10))

# Optional: save to CSV
out_csv = "predictions_july_excess_regression.csv"
try:
    reg_all.to_csv(out_csv, index=False)
    print(f"Saved regression predictions to {out_csv}")
except Exception as e:
    print("Could not save CSV:", e)


/Users/valentinaovallevelandia/Library/Python/3.9/lib/python/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/valentinaovallevelandia/Library/Python/3.9/lib/python/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/valentinaovallevelandia/Library/Python/3.9/lib/python/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/Users/valentinaovallevelandia/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_base.py:279: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/Users/valentinaovallevelandia/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_base.py:279: RuntimeWarning: overflow encountered in matmul
  return X @ coef_ + self.intercept_
/Users/valentinaovallevelandia/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_

Ridge | R2=0.038  MAE=0.07041
   RF | R2=0.837  MAE=0.02574
 HGBR | R2=0.342  MAE=0.06017

July 2023 regression predictions (head):


/Users/valentinaovallevelandia/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_base.py:279: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/Users/valentinaovallevelandia/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_base.py:279: RuntimeWarning: overflow encountered in matmul
  return X @ coef_ + self.intercept_
/Users/valentinaovallevelandia/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_base.py:279: RuntimeWarning: invalid value encountered in matmul
  return X @ coef_ + self.intercept_


,stock_id,month_id,ridge,rf,hgbr,mean_pred_excess
0,US001,2023-07,-0.006927,-0.008928,-0.002686,-0.006180
1,US002,2023-07,-0.010202,-0.011823,-0.007136,-0.009720
2,US003,2023-07,-0.011089,0.007046,-0.004731,-0.002925
3,US004,2023-07,-0.013252,0.005502,-0.008745,-0.005498
4,US005,2023-07,-0.001196,-0.002000,0.002934,-0.000087
5,US006,2023-07,-0.002106,0.013226,0.006378,0.005833
6,US007,2023-07,0.014726,0.035959,0.013013,0.021233
7,US008,2023-07,-0.020159,0.019727,-0.006675,-0.002369
8,US009,2023-07,-0.016079,0.003566,-0.003456,-0.005323
9,US010,2023-07,-0.014203,0.006606,-0.010456,-0.006018


Saved regression predictions to predictions_july_excess_regression.csv


In [66]:
# ===== Analysis Task 2.2: Feature drivers, correlations, sector patterns, outliers =====
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score

# Helper: get feature names after preprocessor
feat_names = list(pipe_rf.named_steps["pre"].get_feature_names_out()) if 'pipe_rf' in globals() else (
    list(pipe_lr.named_steps["pre"].get_feature_names_out()) if 'pipe_lr' in globals() else feat_cols
)

# 1) Feature importance across models
analyses = {}

# Logistic Regression coefficients (absolute magnitude as importance proxy)
try:
    coefs = pipe_lr.named_steps['clf'].coef_.ravel()
    imp_lr = (pd.DataFrame({'feature': feat_names, 'importance': np.abs(coefs), 'coef': coefs})
                .sort_values('importance', ascending=False)
                .head(25))
    analyses['lr_top'] = imp_lr
    print("Top 25 logistic-coefficient features (abs magnitude):")
    display(imp_lr)
except Exception as e:
    print('LR importances not available:', e)

# Random Forest feature importances
try:
    rf_feats = list(pipe_rf.named_steps['pre'].get_feature_names_out())
    rf_imps  = pipe_rf.named_steps['clf'].feature_importances_
    imp_rf = (pd.DataFrame({'feature': rf_feats, 'importance': rf_imps})
                .sort_values('importance', ascending=False)
                .head(25))
    analyses['rf_top'] = imp_rf
    print("Top 25 RF feature importances:")
    display(imp_rf)
except Exception as e:
    print('RF importances not available:', e)

# HistGradientBoosting feature importances (if exposed)
try:
    hgb_feats = list(pipe_hgb.named_steps['pre'].get_feature_names_out())
    hgb_imps  = getattr(pipe_hgb.named_steps['clf'], 'feature_importances_', None)
    if hgb_imps is not None:
        imp_hgb = (pd.DataFrame({'feature': hgb_feats, 'importance': hgb_imps})
                     .sort_values('importance', ascending=False)
                     .head(25))
        analyses['hgb_top'] = imp_hgb
        print("Top 25 HGB feature importances:")
        display(imp_hgb)
except Exception as e:
    print('HGB importances not available:', e)

# 2) Correlation analysis with the benchmark index and target
try:
    # Ensure we have macro with index_return and train_df with excess_return
    corr_df = train_df[['stock_id','month_id','excess_return']].copy()
    corr_df = corr_df.join(X_train.reset_index(drop=True))
    corr_df = corr_df.merge(macro[['month_id','index_return']], on='month_id', how='left')

    # Correlations of numeric features with excess_return and with index_return
    num_cols = [c for c in corr_df.columns if c not in ['stock_id','month_id'] and np.issubdtype(corr_df[c].dtype, np.number)]
    corr_excess = corr_df[num_cols].corrwith(corr_df['excess_return']).sort_values(ascending=False)
    corr_index  = corr_df[num_cols].corrwith(corr_df['index_return']).sort_values(ascending=False)

    print("Top correlations with excess_return (train):")
    display(corr_excess.head(15).to_frame('corr_with_excess'))
    print("\nMost negative correlations with excess_return (train):")
    display(corr_excess.tail(15).to_frame('corr_with_excess'))

    print("\nTop correlations with index_return (same-month, train):")
    display(corr_index.head(15).to_frame('corr_with_index'))
    print("\nMost negative correlations with index_return (train):")
    display(corr_index.tail(15).to_frame('corr_with_index'))
except Exception as e:
    print('Correlation analysis failed:', e)

# 3) Sector-based performance patterns across market regimes
try:
    # Build training view with sector and targets
    tdf = train_df[['stock_id','month_id','sector','excess_return']].copy()
    tdf['hit'] = (train_df['outperform_binary_x'].astype(int) if 'outperform_binary_x' in train_df.columns else train_df['outperform_binary'].astype(int))
    tdf = tdf.merge(macro[['month_id','index_return','vix']], on='month_id', how='left')

    # Define regimes by index terciles
    q_lo, q_hi = tdf['index_return'].quantile([0.33, 0.67])
    def regime_index(r):
        if r <= q_lo: return 'bear'
        if r >= q_hi: return 'bull'
        return 'neutral'
    tdf['mkt_regime'] = tdf['index_return'].apply(regime_index)

    # Sector stats overall
    by_sector = (tdf.groupby('sector')
                   .agg(hit_rate=('hit','mean'),
                        mean_excess=('excess_return','mean'),
                        n=('hit','size'))
                   .sort_values(['hit_rate','mean_excess'], ascending=False))
    print("Sector performance — overall (2020-01 to 2023-06):")
    display(by_sector)

    # Sector stats by regime
    by_sector_regime = (tdf.groupby(['mkt_regime','sector'])
                          .agg(hit_rate=('hit','mean'),
                               mean_excess=('excess_return','mean'),
                               n=('hit','size'))
                          .reset_index())
    print("\nSector performance by market regime (bear/neutral/bull):")
    display(by_sector_regime.head(30))

    # Simple time-slice: 2020 vs 2021-2023
    tdf['year'] = tdf['month_id'].dt.year.astype(int)
    by_year_sector = (tdf.groupby(['year','sector'])
                        .agg(hit_rate=('hit','mean'),
                             mean_excess=('excess_return','mean'),
                             n=('hit','size'))
                        .reset_index())
    print("\nSector performance by year:")
    display(by_year_sector.head(30))
except Exception as e:
    print('Sector analysis failed:', e)

# 4) Outlier diagnostics (influential samples)
try:
    # Classification: confident misclassifications by LR (on train)
    p_tr = pipe_lr.predict_proba(X_train)[:,1]
    y_tr = train_df['outperform_binary_x'].astype(int) if 'outperform_binary_x' in train_df.columns else train_df['outperform_binary'].astype(int)
    pred = (p_tr >= 0.5).astype(int)
    conf = np.abs(p_tr - 0.5) * 2  # 0..1 confidence proxy
    mis = (pred != y_tr)
    out_cls = (train_df.loc[mis, ['stock_id','month_id','sector']]
                        .assign(prob=p_tr[mis], conf=conf[mis], true=y_tr[mis], pred=pred[mis])
                        .sort_values('conf', ascending=False)
                        .head(15))
    print("Top 15 confident misclassifications (LR):")
    display(out_cls)

    # Regression: train residuals (HGBR)
    if 'pipe_hgbr' in globals():
        y_reg = train_df['excess_return'].astype(float)
        pred_reg = pipe_hgbr.predict(X_train)
        resid = y_reg - pred_reg
        out_reg = (train_df[['stock_id','month_id','sector']]
                     .assign(y=y_reg, yhat=pred_reg, resid=resid)
                     .assign(abs_resid=lambda d: d['resid'].abs())
                     .sort_values('abs_resid', ascending=False)
                     .head(15))
        print("\nTop 15 regression residual outliers (HGBR):")
        display(out_reg)
except Exception as e:
    print('Outlier diagnostics failed:', e)


Top 25 logistic-coefficient features (abs magnitude):


,feature,importance,coef
13,5y_treasury_lag1,0.523750,0.523750
14,inflation_rate_lag1,0.493033,-0.493033
12,10y_treasury_lag1,0.474786,0.474786
15,fed_rate_lag1,0.451490,-0.451490
22,10y_treasury_CHG_1M_lag1,0.402639,-0.402639
23,5y_treasury_CHG_1M_lag1,0.346699,0.346699
11,vix_lag1,0.264590,0.264590
19,TERM_SPREAD_lag1,0.233554,0.233554
17,index_return_lag1,0.192217,-0.192217
25,fed_rate_CHG_1M_lag1,0.185402,-0.185402


Top 25 RF feature importances:


,feature,importance
1,intramonth_volatility_lag1,0.068487
5,return_3m_lag1,0.065601
6,return_6m_lag1,0.064612
3,price_range_ratio_lag1,0.063026
9,volatility_6m_lag1,0.060934
4,return_1m_lag1,0.060618
8,volatility_3m_lag1,0.057984
10,volume_ratio_lag1,0.056526
2,monthly_volume_lag1,0.054333
0,avg_volume_3m_lag1,0.054151


Correlation analysis failed: You are trying to merge on period[M] and object columns for key 'month_id'. If you wish to proceed you should use pd.concat
Sector analysis failed: You are trying to merge on period[M] and object columns for key 'month_id'. If you wish to proceed you should use pd.concat
Top 15 confident misclassifications (LR):


/Users/valentinaovallevelandia/Library/Python/3.9/lib/python/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/valentinaovallevelandia/Library/Python/3.9/lib/python/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/valentinaovallevelandia/Library/Python/3.9/lib/python/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


,stock_id,month_id,sector,prob,conf,true,pred
23991,US598,2020-05,Energy,0.759194,0.518389,0,1
1505,US037,2020-05,Energy,0.748337,0.496674,0,1
22905,US566,2020-03,Healthcare,0.268991,0.462018,1,0
22946,US567,2020-02,Healthcare,0.272535,0.454929,1,0
6125,US148,2020-04,Real_Estate,0.727429,0.454859,0,1
23862,US595,2020-02,Industrials,0.273081,0.453837,1,0
22904,US566,2020-02,Healthcare,0.274413,0.451175,1,0
20798,US503,2020-04,Technology,0.721278,0.442556,0,1
19118,US462,2020-04,Industrials,0.718584,0.437167,0,1
18404,US445,2020-04,Consumer_Discretionary,0.718484,0.436967,0,1



Top 15 regression residual outliers (HGBR):


,stock_id,month_id,sector,y,yhat,resid,abs_resid
23990,US598,2020-04,Energy,3.115830,0.478860,2.636971,2.636971
24032,US599,2020-04,Energy,2.370746,1.033977,1.336769,1.336769
25087,US633,2020-07,Consumer_Discretionary,1.867838,0.548166,1.319673,1.319673
14883,US360,2020-11,Technology,1.551027,0.235868,1.315159,1.315159
1504,US037,2020-04,Energy,2.105590,1.006820,1.098770,1.098770
22686,US555,2020-12,Healthcare,1.260515,0.170300,1.090215,1.090215
24039,US599,2020-11,Energy,1.596306,0.538073,1.058232,1.058232
22117,US540,2021-01,Finance,0.993841,-0.001695,0.995536,0.995536
22920,US566,2021-06,Healthcare,1.132868,0.196601,0.936266,0.936266
22820,US564,2020-01,Healthcare,0.916226,-0.005147,0.921372,0.921372


In [ ]:
# ===== Evaluation Metrics: F1 (Classification) and RMSE (Regression) =====
# We evaluate on a validation month (2023-06), since July 2023 has no labels.
from sklearn.metrics import f1_score, precision_score, recall_score, mean_squared_error
import numpy as np
import pandas as pd

VAL_MONTH = TRAIN_END  # 2023-06
prev_m = VAL_MONTH - 1

# Build features for validation month from prev month (lag=1 alignment)
stock_prev_v = stock.loc[stock["month_id"] == prev_m, ['stock_id'] + to_lag].copy()
stock_prev_v = stock_prev_v.rename(columns={c: f'{c}_lag1' for c in to_lag})
stock_prev_v['month_id'] = VAL_MONTH

macro_prev_v = macro.loc[macro["month_id"] == prev_m].copy()
macro_prev_v = macro_prev_v.rename(columns={c: f'{c}_lag1' for c in macro_prev_v.columns if c != 'month_id'})
macro_prev_v['month_id'] = VAL_MONTH

panel_val = (stock_prev_v
             .merge(macro_prev_v, on='month_id', how='left')
             .merge(company, on='stock_id', how='left'))

val_df = (train_tgt.loc[train_tgt['month_id'] == VAL_MONTH, ['stock_id','month_id','outperform_binary','excess_return']]
          .merge(panel_val, on=['stock_id','month_id'], how='left'))

# Use same feature set as training
X_val = val_df.reindex(columns=X_train.columns).replace([np.inf, -np.inf], np.nan)
X_val = X_val.fillna(train_medians)

# Classification targets
y_val = val_df['outperform_binary'].astype(int)

# Helper to evaluate a classifier

def eval_clf(name, pipe):
    proba = pipe.predict_proba(X_val)[:,1]
    pred = (proba >= 0.5).astype(int)
    f1  = f1_score(y_val, pred)
    prec = precision_score(y_val, pred)
    rec  = recall_score(y_val, pred)
    print(f"{name:>6} | F1={f1:.4f}  Precision={prec:.4f}  Recall={rec:.4f}  n={len(y_val)}")

print("Validation month:", str(VAL_MONTH))
try:
    eval_clf("LR",  pipe_lr)
    eval_clf("RF",  pipe_rf)
    eval_clf("HGB", pipe_hgb)
except Exception as e:
    print("Classification eval skipped:", e)

# Regression evaluation: RMSE on continuous excess_return
try:
    if 'pipe_ridge' in globals() and 'pipe_rfr' in globals() and 'pipe_hgbr' in globals():
        yv = val_df['excess_return'].astype(float)
        for name, pipe in [("Ridge", pipe_ridge),("RFReg", pipe_rfr),("HGBR", pipe_hgbr)]:
            pred = pipe.predict(X_val)
            rmse = mean_squared_error(yv, pred, squared=False)
            print(f"{name:>6} | RMSE={rmse:.6f}  n={len(yv)}")
    else:
        print("Regression models not available; run Advanced Task cell first.")
except Exception as e:
    print("Regression eval skipped:", e)
